In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in feature sets and corresponding outputs
# Some values of a_max were too large for a 64-bit number,
# so a 128-bit float had to be specified in order for the
# column to be parsed correctly (otherwise Pandas defaulted
# to parsing them as strings)
X1 = pd.read_csv("features_idl.csv")
X2 = pd.read_csv("features_lia.csv",dtype={'a_max':np.float128})
y1 = pd.read_csv("best_solver_idl.csv")
y2 = pd.read_csv("best_solver_lia.csv")

In [3]:
# Convert output values to 0 for cvc4, 1 for z3, or 2 for sat
y1 = y1.values
y1 = pd.DataFrame(np.where(y1 == "sat", 2, np.where(y1 == "z3", 1, np.where(y1 == "cvc4", 0, -1))))

y2 = y2.values
y2 = pd.DataFrame(np.where(y2 == "sat", 2, np.where(y2 == "z3", 1, np.where(y2 == "cvc4", 0, -1))))

# Verifies that there were no values in the data other than "cvc4", "z3", or "sat"
assert(not -1 in y1.values)
assert(not -1 in y2.values)

In [4]:
# Combine data from IDL and LIA datasets
X = pd.concat([X1,X2])
y = pd.concat([y1,y2])

In [5]:
from sklearn.model_selection import train_test_split

# Split datasets into
# training (60%)
# validation (20%)
# testing (20%)
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25, random_state=1)

In [6]:
# Combine output and features
train = pd.concat([y_train, X_train], axis=1)
val = pd.concat([y_val, X_val], axis=1)
test = pd.concat([y_test, X_test], axis=1)

In [7]:
train.to_csv('train.csv', index=False, header=False)
val.to_csv('validation.csv', index=False, header=False)
test.to_csv('test.csv', index=False, header=False)

In [8]:
import sagemaker, boto3, os
bucket = sagemaker.Session().default_bucket()
prefix = "smt-eager-vs-lazy"

# Upload datasets to S3
boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/test.csv')).upload_file('test.csv')

In [9]:
region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()

In [13]:
from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput

# Configure model

s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[Rule.sagemaker(rule_configs.create_xgboost_report())]
)

xgb_model.set_hyperparameters(
    objective = 'multi:softprob',
    num_class = 3,
    num_round = 100,
    subsample = 0.7,
    colsample_bytree = 0.8
)

257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.2-1


In [14]:
from sagemaker.session import TrainingInput

train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "data/train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "data/validation.csv"), content_type="csv"
)

In [15]:
# Run the training job to fit the model
xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)

2021-08-17 14:15:12 Starting - Starting the training job...
2021-08-17 14:15:35 Starting - Launching requested ML instancesCreateXgboostReport: InProgress
ProfilerReport-1629209712: InProgress
...
2021-08-17 14:16:02 Starting - Preparing the instances for training.........
2021-08-17 14:17:36 Downloading - Downloading input data...
2021-08-17 14:18:04 Training - Downloading the training image...
2021-08-17 14:18:44 Training - Training image download completed. Training in progress..[2021-08-17 14:18:43.456 ip-10-0-173-219.us-east-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softprob to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined del

In [16]:
# Get the auto-generated analytics
rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive

2021-08-17 14:20:10     322345 smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-report.html
2021-08-17 14:20:10     168689 smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-report.ipynb
2021-08-17 14:20:06        190 smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-reports/BatchSize.json
2021-08-17 14:20:06        198 smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-reports/CPUBottleneck.json
2021-08-17 14:20:06        126 smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-reports/Dataloader.json
2021-08-17 14:20:06        127 smt-eager-vs-lazy/xgboost_mode

In [17]:
# Download the auto-generated analytics
! aws s3 cp {rule_output_path} ./ --recursive

download: s3://sagemaker-us-east-2-736959812641/smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-reports/BatchSize.json to ProfilerReport-1629209712/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-2-736959812641/smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-report.html to ProfilerReport-1629209712/profiler-output/profiler-report.html
download: s3://sagemaker-us-east-2-736959812641/smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712/profiler-output/profiler-reports/CPUBottleneck.json to ProfilerReport-1629209712/profiler-output/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-us-east-2-736959812641/smt-eager-vs-lazy/xgboost_model/sagemaker-xgboost-2021-08-17-14-15-12-406/rule-output/ProfilerReport-1629209712

In [18]:
# When done training/tuning the model, deploy an endpoint to SageMaker
import sagemaker
from sagemaker.serializers import CSVSerializer
xgb_predictor=xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    serializer=CSVSerializer()
)

----------!

In [19]:
import numpy as np

# This function calls the endpoint to get predictions
# from the model and processes the returned data
def predict_multi_class(data, num_class, rows=1000):
    
    assert(num_class >= 2)
    
    num_examples = data.shape[0]
    
    split_array = np.array_split(data, int(num_examples / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    
    # For binary classifiers, predict() returns a single float:
    # the probability of a positive outcome
    # formally, this means the model returns 1
    if num_class == 2:
        return np.fromstring(predictions[1:], sep=',')
    
    # Convert string version of 2D array to Python list of strings
    pred_list = predictions[1:].replace('[','').replace(']','').strip().split(',')
    
    try:
        assert(len(pred_list) == num_examples * num_class)
    except AssertionError:
        print("Something went wrong. Verify that the value of num_class is correct.")
        exit()
    
    # Convert Python list to Numpy array of floats, and reshape to 2D
    return np.array(pred_list, dtype=float).reshape([num_examples,num_class])

In [21]:
import sklearn

# Output the accuracy of the model on the test set
log_predictions = predict_multi_class(test.to_numpy()[:,1:], 3)
predictions = np.argmax(log_predictions, axis=1)
sklearn.metrics.accuracy_score(test.iloc[:,0], predictions)

0.75

In [22]:
# Output the confusion matrix for the test set
cm = sklearn.metrics.confusion_matrix(test.iloc[:,0], predictions)
cm

array([[ 5,  2,  0],
       [ 4, 23,  1],
       [ 1,  2,  2]])

In [28]:
np.count_nonzero(np.where(y_train == 0))

20

In [29]:
np.count_nonzero(np.where(y_train == 1))

86

In [30]:
np.count_nonzero(np.where(y_train == 2))

10